# Transport

Ratio par jour à partir des modes de transport + distance des cadres

- Moyenne distance travail cadre: 19km [Observatoire des territoires](https://www.observatoire-des-territoires.gouv.fr/kiosque/2019-mobilite-10-les-cadres-parcourent-chaque-annee-pres-de-3000-km-de-plus-que-les)
- Ratio Déplacements cadres et professions intellectuelles, toutes activités dans un rayon de 80km: [INSEE](https://www.statistiques.developpement-durable.gouv.fr/resultats-detailles-de-lenquete-mobilite-des-personnes-de-2019?rubrique=60&dossier=1345)
    1. Marche à pied: 1.3818
    2. Vélo: 1.44675
    3. Transport en commun: 17.77558
    4. Voiture: 76.32407
    5. Deux roues motorisé: 2.91722
    6. Autre: 0.15459

- Emissions transports:
    - Voiture - Motorisation moyenne - 2018: 
        - 0.218 kg CO2-eq [ADEME](https://bilans-ges.ademe.fr/fr/basecarbone/donnees-consulter/liste-element/categorie/151)
    - Vélo
        - Moyenne fabrication [Trek](https://view.publitas.com/trek-bicycle/trek-bicycle-2021-sustainability-report/page/5): 174kg CO2
        - 50 000km (voir notes)
    - Métro, tramway, trolleybus - 2018 - Agglomération de 100 000 à 250 000 habitants
        - 0.00503 kgCO2e/passager.km [ADEME](https://bilans-ges.ademe.fr/fr/accueil/documentation-gene/index/page/Ferroviaire2)
    - Moto > 250 cm3 - Mixte - 2018 
        - 0.191 kgCO2e/km [ADEME](https://bilans-ges.ademe.fr/fr/accueil/documentation-gene/index/page/Routier2)


In [1]:
CAR_CO2_KM = 0.218
BIKE_CO2_KM = 174 / 50000  # 0,00348
PUBLIC_TRANSPORT_CO2_KM = 0.00503
MOTORBIKE_CO2_KM = 0.191

FOOT_PERCENTAGE = 1.3818
BIKE_PERCENTAGE = 1.44675
PUBLIC_TRANSPORT_PERCENTAGE = 17.77558
CAR_PERCENTAGE = 76.32407 + 0.15459  # other categories (tractors...)
MOTORBIKE_PERCENTAGE = 2.91722

AVG_KM_DAY = 19 * 2

mixed_emissions_km = (
                             FOOT_PERCENTAGE * 0
                             + BIKE_PERCENTAGE * BIKE_CO2_KM
                             + PUBLIC_TRANSPORT_PERCENTAGE * PUBLIC_TRANSPORT_CO2_KM
                             + CAR_PERCENTAGE * CAR_CO2_KM
                             + MOTORBIKE_PERCENTAGE * MOTORBIKE_CO2_KM
                     ) / 100

transport_co2_person = mixed_emissions_km * AVG_KM_DAY
print(transport_co2_person)

6.583113447812


# Buildings

- Surface par salarié: 18 m2/poste avec un taux de bureaux de 70% [Legifrance](https://www.legifrance.gouv.fr/jorf/id/JORFTEXT000042994780)
- ACV bureaux: 3,9 t eqCO2/m2 [Observatoire de l'immobilier durable](https://resources.taloen.fr/resources/documents/7765_191210_poids_carbone_ACV_vdef.pdf)
    - 2,3t travaux
    - 1,6t exploitation
    - 50 ans d'utilisation

In [2]:
OFFICE_EMISSIONS_SQR_METER_DAY = 3900 / (50 * 365)  # 0,21
# 18m2/office are 70% of the building
office_co2_person = ((18 * 100) / 70) * OFFICE_EMISSIONS_SQR_METER_DAY

print(office_co2_person)

5.495107632093934


# Servers

- Electricité - 2020 - mix moyen - consommation: 0.0599 kgCO2e/kWh [ADEME](https://bilans-ges.ademe.fr/fr/accueil/documentation-gene/index/page/Electricite_reglementaire)
- Serveur [Boavitza](https://github.com/Boavizta/environmental-footprint-data)
    - Fabrication: 1613.25
    - Life expectency: 3.89
    - Conso RUN = 1100 (Orange) 
    - Conso IDLE = 234 (Orange)

- Réseau:
    - 1,5g / GO, (Orange)

In [3]:
ELECTRICITY_MIX = 0.0599

SERVER_POWER_IDLE = 234
SERVER_POWER_RUN = 1100
SERVER_LIFE = 3.89
SERVER_FABRICATION_CO2 = 1613.25
SERVER_USAGE  = 0.7
PUE = 1.5

amortization_day = SERVER_FABRICATION_CO2 / (SERVER_LIFE * 365)

wh = (SERVER_POWER_RUN - SERVER_POWER_IDLE) * SERVER_USAGE + SERVER_POWER_IDLE
wh_pue = wh * PUE
kwh_day = (wh_pue * 24) / 1000
server_day = kwh_day * ELECTRICITY_MIX + amortization_day
print(server_day)

2.9480188516448917


# Disks

- 250 kg CO2-eq pour un disque 4TO
- 4 ans (Orange)
- 0,89 W/h HDD, 1,52 W/h SSD pour 1 TO (Orange)

In [4]:
HDD_TB_CONS = 0.89
SSD_TB_CONS = 1.52

DISK_SIZE = 4
DISK_LIFE = 4
DISK_FABRICATION_CO2 = 250

wh_to = SSD_TB_CONS
wh_disk = DISK_SIZE * wh_to
wh_pue = wh_disk * PUE
kwh_day = (wh_pue * 24) / 1000
disk_day = kwh_day * ELECTRICITY_MIX

print(disk_day)

0.013110912000000002


# Users

- Smartphone: 
    - Par jour: 
        - 3,12h [Article](https://ieeexplore.ieee.org/abstract/document/6360448)
        - 2,79 [Article](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7960745/)

- Ordinateur portable :
    - [ADEME](https://bilans-ges.ademe.fr/documentation/UPLOAD_DOC_FR/index.htm?ordinateurs_et_equuipements_pe.htm):
        - 169 kg CO2-eq 
        - 5,6 ans [EU](https://www.eup-network.de/fileadmin/user_upload/Produktgruppen/Lots/Final_Documents/EuP_Lot3_PC_FinalReport.pdf)
    - [Boavitza](https://github.com/Boavizta/environmental-footprint-data)
        - 307.37 kg
        - 4.34 ans

Ratio 50/50 accès smartphone/laptop

In [1]:
SMARTPHONE_CO2 = 88.75
SMARTPHONE_LIFE = 2
SMARTPHONE_DAILY_USE = 3.12

LAPTOP_CO2 = 307.37
LAPTOP_LIFE = 4.34
PC_DAILY_USE = 7


smartphone_day_co2 = SMARTPHONE_CO2 / (SMARTPHONE_LIFE * 365)
smartphone_hour_co2 = smartphone_day_co2 / SMARTPHONE_DAILY_USE

laptop_day_co2 = LAPTOP_CO2 / (LAPTOP_LIFE * 365)
laptop_hour_co2 = laptop_day_co2 / PC_DAILY_USE

user_hour_co2 = laptop_hour_co2 * 0.5 + smartphone_hour_co2 * 0.5

print(user_hour_co2)

0.0333428327821832
